In [1]:
import pandas as pd 
import geopandas as gpd
import numpy as np
import time
from tqdm import tqdm

/Users/lorenzo/opt/anaconda3/lib/python3.9/site-packages/pandas/core/computation/expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.8.3' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
/Users/lorenzo/opt/anaconda3/lib/python3.9/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [2]:
HST = pd.read_csv("/Volumes/T7/Water Project/npdes_downloads/NPDES_QNCR_HISTORY.csv", 
                 dtype = {'YEARQTR' : str})

In [3]:
HST

,NPDES_ID,YEARQTR,HLRNC,NUME90Q,NUMCVDT,NUMSVCD,NUMPSCH
0,AK0000086,19932,,1,0,0,0
1,AK0000086,19951,,0,0,0,2
2,AK0000086,19952,,0,0,0,2
3,AK0000086,19953,,0,0,0,2
4,AK0000086,19954,,0,0,0,2
...,...,...,...,...,...,...,...
6895200,WVR106895,20182,C,0,0,0,0
6895201,WVR106895,20183,C,0,0,0,0
6895202,WVR106895,20184,C,0,0,0,0
6895203,WVR106895,20191,C,0,0,0,0


In [4]:
twtyears = HST[HST['YEARQTR'].str[:4] > "2003"]

In [5]:
IDS = twtyears["NPDES_ID"].unique().tolist()

In [6]:
len(IDS)

552531

In [7]:

#for i in IDS:
 #   SNC = ["S", "E", "X", "T", "D", "W"]
 #   data = twtyears[twtyears["NPDES_ID"] == i]
 #   print(i)
 #   print(len(data))
 #   print(len(data[data["HLRNC"].isin(SNC)]))
    

In [8]:
TRIAL1 = {"NPDES_ID" : list(), 
        "TOT_QTR_REP": list(), 
        "QTR_IN_SNC" : list()}

SNC = ["S", "E", "X", "T", "D", "W"]

In [9]:
grouped = twtyears.groupby("NPDES_ID")

# Use tqdm to display progress bar
for i, data in tqdm(grouped):
    table = data["HLRNC"].value_counts()
    TRIAL1["NPDES_ID"].append(i)
    TRIAL1["TOT_QTR_REP"].append(table.sum())
    TRIAL1["QTR_IN_SNC"].append(table[table.index.isin(SNC)].sum())

100%|█████████████████████████████████| 552531/552531 [01:39<00:00, 5572.45it/s]


In [10]:
inf_QTR = pd.DataFrame.from_dict(TRIAL1)

In [11]:
inf_QTR["PERCENT"] = 100 * inf_QTR["QTR_IN_SNC"]/inf_QTR["TOT_QTR_REP"]

In [12]:
inf_QTR

,NPDES_ID,TOT_QTR_REP,QTR_IN_SNC,PERCENT
0,AK0000167,8,0,0.000000
1,AK0000272,64,6,9.375000
2,AK0000345,80,0,0.000000
3,AK0000361,10,10,100.000000
4,AK0000370,54,4,7.407407
...,...,...,...,...
552526,WYS000014,33,28,84.848485
552527,WYS000015,16,0,0.000000
552528,WYS000016,7,6,85.714286
552529,WYS000019,3,0,0.000000


In [13]:
Geos = pd.read_csv("/Volumes/T7/Water Project/npdes_downloads/ICIS_FACILITIES.csv")

In [14]:
Geos.columns

Index(['ICIS_FACILITY_INTEREST_ID', 'NPDES_ID', 'FACILITY_UIN',
       'FACILITY_TYPE_CODE', 'FACILITY_NAME', 'LOCATION_ADDRESS',
       'SUPPLEMENTAL_ADDRESS_TEXT', 'CITY', 'COUNTY_CODE', 'STATE_CODE', 'ZIP',
       'GEOCODE_LATITUDE', 'GEOCODE_LONGITUDE', 'IMPAIRED_WATERS'],
      dtype='object')

In [15]:
Geos = Geos[["NPDES_ID", "FACILITY_NAME", "CITY",
             "COUNTY_CODE", "STATE_CODE", "ZIP",
             "GEOCODE_LATITUDE", "GEOCODE_LONGITUDE"]]

In [16]:
merged = inf_QTR.merge(Geos, on = "NPDES_ID", how = "right")

In [17]:
merged

,NPDES_ID,TOT_QTR_REP,QTR_IN_SNC,PERCENT,FACILITY_NAME,CITY,COUNTY_CODE,STATE_CODE,ZIP,GEOCODE_LATITUDE,GEOCODE_LONGITUDE
0,KYR10I122,2.0,0.0,0.00,HAWKINS FILL SITE,SHEPHERDSVILLE,KY029,KY,40165,38.042056,-85.683603
1,LAL036412,16.0,1.0,6.25,E BATON ROUGE CITY-PAR (SOUTH),BATON ROUGE,LA033,LA,70820,30.348889,-91.135556
2,MAU000097,NaN,NaN,NaN,DEVELOPMENT AT 65 SUNNYSIDE AVENUE,HOLDEN,MA027,MA,01520,42.351389,-71.870240
3,PRU201920,NaN,NaN,NaN,DUARTA WASTE REPAIR SHOP,BAYAMON,PR021,PR,00956,NaN,NaN
4,UTR000146,NaN,NaN,NaN,BURDICK PAVING,ROOSEVELT,UT013,UT,84066,40.393778,-110.150083
...,...,...,...,...,...,...,...,...,...,...,...
960792,INRA13134,NaN,NaN,NaN,RETREAT AT MORSE,NOBLESVILLE,NaN,IN,46062,40.073821,-86.036126
960793,INRA13107,NaN,NaN,NaN,AIRPORT EXPY & ARDMORE AVE STORMWATER IMPROVEM...,FORT WAYNE,NaN,IN,46809,41.000232,-85.194721
960794,INRA13146,NaN,NaN,NaN,ADDITIONS AND RENOVATIONS TO CARROLL HIGH SCHOOL,FORT WAYNE,NaN,IN,46818,41.190534,-85.192547
960795,COR413808,NaN,NaN,NaN,NIVER CREEK TRIBUTARY M IMPROVEMENTS,THORNTON,NaN,CO,80260,39.860314,-104.996688


In [18]:
merged.to_csv("/Volumes/T7/Water Project/NPDES_FINAL.csv")